In [5]:
%matplotlib inline

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
import json

In [6]:
fname = 'sp-pos-quot-dep-2021-03-28-19h20-nat3C.csv'
d = pd.read_csv(fname, 
                sep = ';', 
                parse_dates = ['jour'], 
                dtype = {'P': int, 'T': int},
                skiprows = [1])
d.sample(3)

,jour,three_class,P,T,pop
59,2020-05-27,whole,725,45997,6.711500e+07
20,2020-05-18,0-19,85,4711,1.609969e+07
979,2021-01-12,whole,21631,339836,6.711500e+07


In [7]:
d['incidence'] = d['P'] / d['pop'] * 100000
d['positivite'] = d['P'] / d['T'] * 100
d.sample(3)

,jour,three_class,P,T,pop,incidence,positivite
162,2020-06-22,60+,137,18134,1.785758e+07,0.767181,0.755487
440,2020-08-31,0-19,1528,28470,1.609969e+07,9.490865,5.367053
163,2020-06-22,whole,595,48101,6.711500e+07,0.886538,1.236981


In [10]:
three_class = d['three_class'].unique()

In [11]:
tic1 = time.time()
d['incidence hebdo'] = np.zeros(len(d))
for age_class in three_class:
    d2 = d[ (d['three_class'] == age_class) ]
    d['temp'] = d2.apply(lambda x : (d2[ (d2['jour'] <= x['jour']) 
                                                    & (d2['jour'] > x['jour'] - np.timedelta64(1,'W'))]
                                                ['incidence']
                                                .sum()), axis = 1)
    d['incidence hebdo'] = d['incidence hebdo'] + d['temp'].fillna(0)
toc1 = time.time()
print('{:.2f} s'.format(-tic1+toc1))

0.76 s


In [12]:
d = d.drop(columns = ['temp'])
d.sample(3)

,jour,three_class,P,T,pop,incidence,positivite,incidence hebdo
458,2020-09-04,60+,1305,48327,1.785758e+07,7.307820,2.700354,33.817566
632,2020-10-18,0-19,578,5801,1.609969e+07,3.590131,9.963799,157.599290
1025,2021-01-24,20-59,1592,23732,3.315772e+07,4.801295,6.708242,243.949222


In [13]:
fname2 = fname[:-4] + '-processed.csv'
d.to_csv(fname2, sep = ';', index = False)